<a href="https://colab.research.google.com/github/Woocash371/GAA/blob/main/Wyb%C3%B3r_cech_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install deap


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.1 MB/s eta 0:00:00


In [3]:
"""import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from deap import base, creator, tools, algorithms
import random

# Przygotowanie danych
file_path = '/content/california_housing.xlsx'
xls = pd.ExcelFile(file_path)
df = pd.read_excel(xls, sheet_name='Sheet1')
df = df.drop(columns=['Unnamed: 0'])

# Separacja cech (X) i celu (y)
X = df.drop(columns=['MedHouseVal'])
y = df['MedHouseVal']

# Standaryzacja cech
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Podział na zbiory treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Definicja algorytmu genetycznego
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

def evaluate(individual):
    # Zastosowanie maski wyboru cech
    mask = np.array(individual, dtype=bool)
    if not any(mask):
        return (float('inf'),)  # Kara za wybór braku cech

    X_train_masked = X_train[:, mask]
    X_test_masked = X_test[:, mask]

    # Model regresji liniowej
    model = LinearRegression()
    model.fit(X_train_masked, y_train)
    y_pred = model.predict(X_test_masked)
    mae = mean_absolute_error(y_test, y_pred)
    return (mae,)

# Inicjalizacja algorytmu genetycznego
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=X.shape[1])
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)  # Zmiana krzyżowania na krzyżowanie dwupunktowe
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Parametry algorytmu genetycznego
population_size = 100
num_generations = 50
crossover_prob = 0.8
mutation_prob = 0.1

# Tworzenie populacji początkowej
population = toolbox.population(n=population_size)

# Uruchomienie algorytmu genetycznego
algorithms.eaSimple(population, toolbox, cxpb=crossover_prob, mutpb=mutation_prob, ngen=num_generations,
                   stats=None, halloffame=None, verbose=True)

# Wybór najlepszego osobnika
best_individual = tools.selBest(population, k=1)[0]
best_features = np.array(best_individual, dtype=bool)

# Wyświetlenie wybranych cech
selected_features = X.columns[best_features]
print("Najbardziej wpływowe cechy:", selected_features.tolist())"""


'import numpy as np\nimport pandas as pd\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.linear_model import LinearRegression\nfrom sklearn.metrics import mean_absolute_error\nfrom sklearn.model_selection import train_test_split\nfrom deap import base, creator, tools, algorithms\nimport random\n\n# Przygotowanie danych\nfile_path = \'/content/california_housing.xlsx\'\nxls = pd.ExcelFile(file_path)\ndf = pd.read_excel(xls, sheet_name=\'Sheet1\')\ndf = df.drop(columns=[\'Unnamed: 0\'])\n\n# Separacja cech (X) i celu (y)\nX = df.drop(columns=[\'MedHouseVal\'])\ny = df[\'MedHouseVal\']\n\n# Standaryzacja cech\nscaler = StandardScaler()\nX_scaled = scaler.fit_transform(X)\n\n# Podział na zbiory treningowy i testowy\nX_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)\n\n# Definicja algorytmu genetycznego\ncreator.create("FitnessMin", base.Fitness, weights=(-1.0,))\ncreator.create("Individual", list, fitness=creator.FitnessMin)\n\

In [26]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from deap import base, creator, tools, algorithms
import random
from multiprocessing import Pool

def genetic_algorithm(file_path, target_column, model_type='LinearRegression', error_metric='mse', population_size=50, num_generations=20, crossover_prob=0.7, mutation_prob=0.2,
                      keep_best_idividual= False,amount_of_best_individual=1,crossover_type='TwoPoint'):
    """

    Algorytm genetyczny do selekcji cech, mający na celu określenie najbardziej wpływowych cech wpływających na zmienną docelową.

    Parametry:
    file_path (str): Ścieżka do pliku z danymi (format Excel).
    target_column (str): Nazwa kolumny zawierającej zmienną docelową do przewidywania.
    model_type (str): Typ modelu regresji do użycia ('LinearRegression', 'RandomForest', 'SVR'). Domyślnie 'LinearRegression'.
        - 'LinearRegression': Model regresji liniowej, który zakłada liniową zależność między zmiennymi niezależnymi a zmienną zależną. Jest prosty w interpretacji i szybki w obliczeniach.
        - 'RandomForest': Model Random Forest, który jest zespołem drzew decyzyjnych. Charakteryzuje się dużą dokładnością oraz odpornością na nadmierne dopasowanie dzięki agregacji wyników wielu drzew.
        - 'SVR': Support Vector Regression, model oparty na maszynach wektorów nośnych, który jest skuteczny w przypadku nieliniowych zależności w danych.
    error_metric (str): Metryka błędu do użycia ('mse' dla średniego błędu kwadratowego, 'mae' dla średniego błędu bezwzględnego, 'r2' dla współczynnika determinacji R^2, 'evs' dla wskaźnika wyjaśnionej wariancji). Domyślnie 'mse'.
        - 'mse': Średni błąd kwadratowy (Mean Squared Error). Jest to średnia kwadratów różnic między wartościami rzeczywistymi a przewidywanymi. Stosowany do karania dużych błędów.
        - 'mae': Średni błąd bezwzględny (Mean Absolute Error). Jest to średnia wartość bezwzględnych różnic między wartościami rzeczywistymi a przewidywanymi. Stosowany do oceny średniej wielkości błędów.
        - 'r2': Współczynnik determinacji R^2. Mierzy, jaka część wariancji zmiennej zależnej jest wyjaśniana przez model. Im wyższa wartość, tym lepsze dopasowanie modelu.
        - 'evs': Wskaźnik wyjaśnionej wariancji (Explained Variance Score). Mierzy, jaka część wariancji danych jest wyjaśniana przez model. Im wyższa wartość, tym lepsze dopasowanie modelu.
    population_size (int): Liczba osobników w populacji. Domyślnie 50.
    num_generations (int): Liczba generacji do przeprowadzenia algorytmu genetycznego. Domyślnie 20.
    crossover_prob (float): Prawdopodobieństwo krzyżowania między osobnikami. Domyślnie 0.7.
    mutation_prob (float): Prawdopodobieństwo mutacji każdego genu. Domyślnie 0.2.
    crossover_type (str): Typ krzyżowania do użycia ('OnePoint', 'TwoPoint', 'MultiPoint', 'Uniform', 'KRandom'). Domyślnie 'TwoPoint'.
    num_crossover_points (int): Liczba punktów krzyżowania do użycia w przypadku krzyżowania wielopunktowego. Domyślnie 2.
    mutation_type (str): Typ mutacji do użycia ('FlipBit', 'ShuffleIndexes', 'Swap', 'Gaussian', 'UniformInt'). Domyślnie 'FlipBit'.
    k_random_points (int): Liczba losowych punktów krzyżowania w przypadku krzyżowania k-randomowego. Domyślnie 3.

    Zwraca:
    None

    Funkcja drukuje najbardziej wpływowe cechy oraz błąd najlepszego osobnika znalezionego przez algorytm genetyczny.
    """


    print(f'Analiza cech dla {target_column}')
    print(f'Typ modelu: {model_type}')
    print(f'Metryka błędu: {error_metric}')
    print(f'Rozmiar populacji: {population_size}')
    print(f'Liczba generacji: {num_generations}')
    print(f'Prawdopodobieństwo krzyżowania: {crossover_prob}')
    print(f'Prawdopodobieństwo mutacji: {mutation_prob}')
    # Przygotowanie danych
    xls = pd.ExcelFile(file_path)
    df = pd.read_excel(xls, sheet_name=0)  # Wczytywanie pierwszego arkusza
    # Usunięcie nienazwanych kolumn
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    # Separacja cech (X) i celu (y)
    X = df.drop(columns=[target_column])
    y = df[target_column]

    # Standaryzacja cech
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Podział na zbiory treningowy i testowy
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Definicja algorytmu genetycznego
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    def evaluate(individual):
        # Zastosowanie maski wyboru cech
        mask = np.array(individual, dtype=bool)
        if not any(mask):
            return (float('inf'),)  # Kara za wybór braku cech

        X_train_masked = X_train[:, mask]
        X_test_masked = X_test[:, mask]

        # Wybór modelu regresji
        if model_type == 'LinearRegression':
            model = LinearRegression()
        elif model_type == 'RandomForest':
            model = RandomForestRegressor(random_state=42)
        elif model_type == 'SVR':
            model = SVR()
        else:
            raise ValueError("Nieznany typ modelu: {}".format(model_type))

        # Trenowanie modelu
        model.fit(X_train_masked, y_train)
        y_pred = model.predict(X_test_masked)

        # Wybór metryki błędu
        if error_metric == 'mse':
            error = mean_squared_error(y_test, y_pred)
        elif error_metric == 'mae':
            error = mean_absolute_error(y_test, y_pred)
        else:
            raise ValueError("Nieznana metryka błędu: {}".format(error_metric))

        return (error,)

    pool = Pool()


    # Inicjalizacja algorytmu genetycznego
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=X.shape[1])
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Rejestracja rodzaju krzyżowania
    if crossover_type == 'OnePoint':
        toolbox.register("mate", tools.cxOnePoint)
    elif crossover_type == 'TwoPoint':
        toolbox.register("mate", tools.cxTwoPoint)
    elif crossover_type == 'PMX':
        toolbox.register("mate", tools.cxPartialyMatched)
    elif crossover_type == 'Uniform':
        toolbox.register("mate", tools.cxUniform, indpb=0.5)
    else:
        raise ValueError("Nieznany typ krzyżowania: {}".format(crossover_type))

    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", evaluate)
    toolbox.register("map", pool.map)
    # Definicja statystyk
    stats = tools.Statistics(lambda ind: ind.fitness.values)

    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    stats.register("median", np.median)
    stats.register("var", np.var)

    if keep_best_idividual == True:
      halloffame = tools.HallOfFame(amount_of_best_individual)
    else:
      halloffame = None
    # Tworzenie populacji początkowej
    population = toolbox.population(n=population_size)

    # Uruchomienie algorytmu genetycznego
    algorithms.eaSimple(population, toolbox, cxpb=crossover_prob, mutpb=mutation_prob, ngen=num_generations,
                       stats=stats, halloffame=halloffame, verbose=True)

    pool.close()
    pool.join()
    for i, individual in enumerate(halloffame):
      print(f'Osobnik {i + 1}: {individual}')

      # Wybranie cech najlepszego osobnika
      features = np.array(individual, dtype=bool)

    # Wyświetlenie wybranych cech
      selected_features = X.columns[features]
      print("Najbardziej wpływowe cechy:", selected_features.tolist())

    # Wyświetlenie błędu osobnika
      fitness = individual.fitness.values[0]
      print("Błąd osobnika ({}):".format(error_metric), fitness)
      print()
      print()



In [27]:

file_path = '/content/sample_data/CISNIENIA i TEMPERATURY.xlsx'


genetic_algorithm(file_path, target_column="P1", model_type='LinearRegression', error_metric='mse', population_size=50, num_generations=20, crossover_prob=0.7, mutation_prob=0.2,
                      keep_best_idividual= False, amount_of_best_individual = 1, crossover_type='TwoPoint')

Analiza cech dla P1
Typ modelu: LinearRegression
Metryka błędu: mse
Rozmiar populacji: 50
Liczba generacji: 20
Prawdopodobieństwo krzyżowania: 0.7
Prawdopodobieństwo mutacji: 0.2


/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


AttributeError: Can't pickle local object 'genetic_algorithm.<locals>.evaluate'

In [50]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from deap import base, creator, tools, algorithms
import random
from multiprocessing import Pool

def genetic_algorithm(file_path, target_column, model_type='LinearRegression', error_metric='mse', population_size=50, num_generations=20, crossover_prob=0.7, mutation_prob=0.2,
                      keep_best_idividual= False,amount_of_best_individual=1,crossover_type='TwoPoint', k_random_points = 1,):
    """

    Algorytm genetyczny do selekcji cech, mający na celu określenie najbardziej wpływowych cech wpływających na zmienną docelową.

    Parametry:
    file_path (str): Ścieżka do pliku z danymi (format Excel).
    target_column (str): Nazwa kolumny zawierającej zmienną docelową do przewidywania.
    model_type (str): Typ modelu regresji do użycia ('LinearRegression', 'RandomForest', 'SVR'). Domyślnie 'LinearRegression'.
        - 'LinearRegression': Model regresji liniowej, który zakłada liniową zależność między zmiennymi niezależnymi a zmienną zależną. Jest prosty w interpretacji i szybki w obliczeniach.
        - 'RandomForest': Model Random Forest, który jest zespołem drzew decyzyjnych. Charakteryzuje się dużą dokładnością oraz odpornością na nadmierne dopasowanie dzięki agregacji wyników wielu drzew.
        - 'SVR': Support Vector Regression, model oparty na maszynach wektorów nośnych, który jest skuteczny w przypadku nieliniowych zależności w danych.
    error_metric (str): Metryka błędu do użycia ('mse' dla średniego błędu kwadratowego, 'mae' dla średniego błędu bezwzględnego, 'r2' dla współczynnika determinacji R^2, 'evs' dla wskaźnika wyjaśnionej wariancji). Domyślnie 'mse'.
        - 'mse': Średni błąd kwadratowy (Mean Squared Error). Jest to średnia kwadratów różnic między wartościami rzeczywistymi a przewidywanymi. Stosowany do karania dużych błędów.
        - 'mae': Średni błąd bezwzględny (Mean Absolute Error). Jest to średnia wartość bezwzględnych różnic między wartościami rzeczywistymi a przewidywanymi. Stosowany do oceny średniej wielkości błędów.
        - 'r2': Współczynnik determinacji R^2. Mierzy, jaka część wariancji zmiennej zależnej jest wyjaśniana przez model. Im wyższa wartość, tym lepsze dopasowanie modelu.
        - 'evs': Wskaźnik wyjaśnionej wariancji (Explained Variance Score). Mierzy, jaka część wariancji danych jest wyjaśniana przez model. Im wyższa wartość, tym lepsze dopasowanie modelu.
    population_size (int): Liczba osobników w populacji. Domyślnie 50.
    num_generations (int): Liczba generacji do przeprowadzenia algorytmu genetycznego. Domyślnie 20.
    crossover_prob (float): Prawdopodobieństwo krzyżowania między osobnikami. Domyślnie 0.7.
    mutation_prob (float): Prawdopodobieństwo mutacji każdego genu. Domyślnie 0.2.
    crossover_type (str): Typ krzyżowania do użycia ('OnePoint', 'TwoPoint', 'MultiPoint', 'Uniform', 'KRandom'). Domyślnie 'TwoPoint'.
    num_crossover_points (int): Liczba punktów krzyżowania do użycia w przypadku krzyżowania wielopunktowego. Domyślnie 2.
    mutation_type (str): Typ mutacji do użycia ('FlipBit', 'ShuffleIndexes', 'Swap', 'Gaussian', 'UniformInt'). Domyślnie 'FlipBit'.
    k_random_points (int): Liczba losowych punktów krzyżowania w przypadku krzyżowania k-randomowego. Domyślnie 3.

    Zwraca:
    None

    Funkcja drukuje najbardziej wpływowe cechy oraz błąd najlepszego osobnika znalezionego przez algorytm genetyczny.
    """



    print(f'Analiza cech dla {target_column}')
    print(f'Typ modelu: {model_type}')
    print(f'Metryka błędu: {error_metric}')
    print(f'Rozmiar populacji: {population_size}')
    print(f'Liczba generacji: {num_generations}')
    print(f'Prawdopodobieństwo krzyżowania: {crossover_prob}')
    print(f'Prawdopodobieństwo mutacji: {mutation_prob}')
    print(f'Typ krzyżowania: {crossover_type}')
    #print(f'Typ mutacji: {mutation_type}')
    if crossover_type == 'KRandom':
      print(f'Liczba punktów krzyżowania: {k_random_points}')
    # Przygotowanie danych
    xls = pd.ExcelFile(file_path)
    df = pd.read_excel(xls, sheet_name=0)  # Wczytywanie pierwszego arkusza
    # Usunięcie nienazwanych kolumn
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    # Separacja cech (X) i celu (y)
    X = df.drop(columns=[target_column])
    y = df[target_column]

    # Standaryzacja cech
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Podział na zbiory treningowy i testowy
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Definicja algorytmu genetycznego
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    def evaluate(individual):
        # Zastosowanie maski wyboru cech
        mask = np.array(individual, dtype=bool)
        if not any(mask):
            return (float('inf'),)  # Kara za wybór braku cech

        X_train_masked = X_train[:, mask]
        X_test_masked = X_test[:, mask]

        # Wybór modelu regresji
        if model_type == 'LinearRegression':
            model = LinearRegression()
        elif model_type == 'RandomForest':
            model = RandomForestRegressor(random_state=42)
        elif model_type == 'SVR':
            model = SVR()
        else:
            raise ValueError("Nieznany typ modelu: {}".format(model_type))

        # Trenowanie modelu
        model.fit(X_train_masked, y_train)
        y_pred = model.predict(X_test_masked)

        # Wybór metryki błędu
        if error_metric == 'mse':
            error = mean_squared_error(y_test, y_pred)
        elif error_metric == 'mae':
            error = mean_absolute_error(y_test, y_pred)
        else:
            raise ValueError("Nieznana metryka błędu: {}".format(error_metric))

        return (error,)


    def cxKRandom(ind1, ind2, k):
      points = sorted(random.sample(range(1, len(ind1)), k))
      for i in range(len(points)):
        if i % 2 == 0:
          start = points[i]
          end = points[i + 1] if i + 1 < len(points) else len(ind1)
          ind1[start:end], ind2[start:end] = ind2[start:end], ind1[start:end]
        return ind1, ind2

    # Inicjalizacja algorytmu genetycznego
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=X.shape[1])
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Rejestracja rodzaju krzyżowania
    if crossover_type == 'OnePoint':
        toolbox.register("mate", tools.cxOnePoint)
    elif crossover_type == 'TwoPoint':
        toolbox.register("mate", tools.cxTwoPoint)
    elif crossover_type == 'PMX':
        toolbox.register("mate", tools.cxPartialyMatched)
    elif crossover_type == 'Uniform':
        toolbox.register("mate", tools.cxUniform, indpb=0.5)
    elif crossover_type == 'KRandom':
        toolbox.register("mate", cxKRandom, k=k_random_points)
    else:
        raise ValueError("Nieznany typ krzyżowania: {}".format(crossover_type))

    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", evaluate)

    # Definicja statystyk
    stats = tools.Statistics(lambda ind: ind.fitness.values)

    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    stats.register("median", np.median)
    stats.register("var", np.var)

    if keep_best_idividual == True:
      halloffame = tools.HallOfFame(amount_of_best_individual)
    else:
      halloffame = None
    # Tworzenie populacji początkowej
    population = toolbox.population(n=population_size)

    # Uruchomienie algorytmu genetycznego
    algorithms.eaSimple(population, toolbox, cxpb=crossover_prob, mutpb=mutation_prob, ngen=num_generations,
                       stats=stats, halloffame=halloffame, verbose=True)


    print("\nNajlepsze osobniki:")
    if keep_best_idividual == True:
        for i, individual in enumerate(halloffame):
            print(f'Osobnik {i + 1}:')
            features = np.array(individual, dtype=bool)
            selected_features = X.columns[features]
            print("  - Wybrane cechy:", selected_features.tolist())
            fitness = individual.fitness.values[0]
            print(f"  - Błąd ({error_metric}): {fitness}\n")
    else:
        best_individual = tools.selBest(population, k=1)[0]
        best_features = np.array(best_individual, dtype=bool)

    # Wyświetlenie wybranych cech
        selected_features = X.columns[best_features]
        print("Najbardziej wpływowe cechy:", selected_features.tolist())"""



In [51]:
genetic_algorithm(file_path, target_column='P1', model_type='LinearRegression',
                  error_metric='mse', population_size=50, num_generations=20, crossover_prob=0.7, mutation_prob=0.2,
                      keep_best_idividual= False,amount_of_best_individual=1,crossover_type='KRandom', k_random_points = 2)

Analiza cech dla P1
Typ modelu: LinearRegression
Metryka błędu: mse
Rozmiar populacji: 50
Liczba generacji: 20
Prawdopodobieństwo krzyżowania: 0.7
Prawdopodobieństwo mutacji: 0.2
Typ krzyżowania: KRandom
Liczba punktów krzyżowania: 2


/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg   	std    	min    	max    	median 	var   
0  	50    	361.88	338.409	113.185	1079.55	120.941	114521
1  	44    	151.671	146.221	113.251	914.379	114.867	21380.7
2  	40    	114.128	0.805952	112.69 	116.586	114.023	0.649558
3  	40    	122.051	59.5504 	112.735	538.894	113.421	3546.25 
4  	34    	113.385	0.517351	112.542	115.881	113.251	0.267652
5  	32    	122.342	63.9095 	112.542	569.703	113.212	4084.42 
6  	40    	113.206	0.547037	112.565	115.998	113.074	0.29925 
7  	40    	113.056	0.417168	112.565	114.782	112.954	0.174029
8  	43    	112.815	0.347866	112.472	114.056	112.766	0.121011
9  	41    	112.659	0.28492 	112.451	113.666	112.565	0.0811793
10 	46    	121.617	63.5099 	112.451	566.186	112.481	4033.51  
11 	42    	112.919	1.67224 	112.451	122.122	112.472	2.79639  
12 	46    	112.593	0.406921	112.451	114.317	112.472	0.165585 
13 	43    	122.144	64.377  	112.451	572.627	112.457	4144.4   
14 	35    	112.578	0.454023	112.451	115.248	112.451	0.206137 
15 	40    	112.53 	0.284547	

TypeError: object of type 'NoneType' has no len()